In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
    border-style: groove;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 3 - Naive Bayes
</center>
</h1>
<div class=h1_cell>
<p>

We will look at a competitor to K-NN this week, Naive Bayes. We will continue to work with the tweets so let's bring them in.

</div>

In [2]:
import pandas as pd
import os

week = '2'

home_path =  os.path.expanduser('~')

file_path = '/Documents/datascience_2/table_history/'

file_name = 'tweet_table_w'+week+'.csv'

tweet_table = pd.read_csv(home_path + file_path + file_name)


In [5]:
library_folder = '/Documents/datascience_2/code_libraries'
os.chdir(home_path + library_folder)
!git pull origin master

From https://github.com/fickas/datascience_2
 * branch            master     -> FETCH_HEAD
Already up to date.


In [6]:
import sys
sys.path.append(home_path + library_folder)
from week2 import *
from mylibrary import *  #I am keeping a private library of some functions
%who function

count_all_hashes	 do_nothing	 foo	 knn_voting	 pat_count	 python_version	 top_k	 


In [25]:
tweet_table.head()

,id,label,tweet,length,bang_count,hash_count,#trump,#politics,#allahsoil,#libtard,#liberal
0,1,0,@user when a father is dysfunctional and is so...,101,0,1,0,0,0,0,0
1,2,0,@user @user thanks for #lyft credit i can't us...,122,0,3,0,0,0,0,0
2,3,0,bihday your majesty,19,0,0,0,0,0,0,0
3,4,0,#model i love u take with u all the time in ...,84,3,1,0,0,0,0,0
4,5,0,factsguide: society now #motivation,38,0,1,0,0,0,0,0


<h2>
Another method: Naive Bayes
</h2>
<div class=h1_cell>
<p>
The problem is still the same. Given a tweet, predict whether it is hate or nohate. The twist is that I am going to reform the question in terms of probabilities using a method called *Naive Bayes*. I'll tell you what is naive about it later. Fow now, you need to do a perspective shift. We will no longer be using a table-based approach that looks at columns of features. Instead we will use counts of things. First, here is the formula we want to compute.
<p>
<img src='https://www.dropbox.com/s/gstzvvtvh9b39o8/bayes.png?raw=1'>
<p>
The way we will use this formula is as follows:
<p>
<ol>
<p>
<li>We have a tweet T with zero or more hashtags. We want to know whether to mark it as hate (1) or nohate (0).</li>
<p>
<li>We compute P(nohate|hashtags). What is the probability that T is nohate given its hashtags.</li>
<p><li>We compute P(hate|hashtags). What is the probability that T is hate given its hashtags.</li>
<p><li>We compare the 2 probabilities. The higher probability wins and we use the associated value for prediction.</li>
</ol>
<p>
In summary, we always compute the probability for all possible prediction values. We only have two, hate and nohate, so we always compute two. If you had three possible values (next week), you would compute three probabilities, etc.
<p>
We read the left hand side as "The probability of value O (hate or nohate) given the evidence which we supply as the hashtags of the tweet. For the right hand side, the notation is as follows:
<ul>
<p>
<li>O stands for one of the prediction values. In our case 0 (nohate) or 1 (hate).</li>
<p><li>E1 ... En stands for the evidence we have for making prediction O. I am going to use the #hashtags we find in the tweet as the evidence. So E1 could be #trump, E2 could be #cnn, etc.</li>
<p><li>P(E1|O) stands for the probability we see E1 (one of the hashtags in the tweet) associated with the case O. This ends up being the total number of times we see E1 with O divided by the total number of O tweets. If the hashtag #trump appears in every hate tweet, then P(#trump|hate) = 1. If it never appears in a hate tweet, then P(#trump|hate) is 0.</li>
<p><li>P(O) is what percentage of tweets that are case O.</li>
<p><li>P(E) = P(E1) `*` P(E2) ... `*` P(En). P(E1) is how many tweets does E1 appear in divided by the total number of tweets. Ditto for P(E2), etc. But guess what? We are going to ignore the denomominator! Wait. You can't just throw out the denominator. But we can. You will see why shortly.
</ul>
<p>
There is a special case we need to deal with. What if the tweet we are trying to predict on has no hashtags? Then we have something that looks like `P(O|)`. Kind of weird. I am going to define that as evidence E0, i.e., P(O|E0).
<p>
Let's think a bit about this. What data do we need to compute the various components of the formula:
<ul>
<p>
<p><li>For P(E1|O) we need to know how many times the hashtag appears in nonhate tweets and how many times it appears in hate tweets. We can add those to get the total number of tweets it appears in.</li>
<p><li>For P(O) we need the count of each type of tweet, nohate and hate. And we can add those to get the total number of tweets.</li>
<p><li>For P(E0|O) we need the count of nohate tweets that have no hashtag and the count of hate tweets that have no hashtag. And we can add those to get the total.
</ul>
<p>
The good news is that we have a large part of this from week 2. That's why I asked you to build the somewhat odd dictionary `all_hashes` in week 2. It contains the info we need for P(Ei|O). Let's bring that in now and get it set up.
</div>

<h2>
Bring over count_all_hashes from week 2
</h2>
<div class=h1_cell>
<p>
Reminder: we need a count on all hashtags broken out by how many times each appears in a nohate tweet and how many times in a hate tweet. So a dictionary with keys as hashes and values as a 2 place list, e.g., `{'#foo': [2, 5], 'fum': [23, 0], ...}.` In this example, we can see that the hash #foo appears 2 times in a nohate tweet and 5 times in a hate tweet.
<p>
You should bring over the `count_all_hashes` function from week 2. And fill it in and execute it.
</div>

In [8]:
import re

In [9]:
#def count_all_hashes(table):
# already imported 
count_all_hashes


<function mylibrary.count_all_hashes>

In [10]:
all_hashes = count_all_hashes(tweet_table)  # this gives us info on P(Ei|O)

In [11]:
len(all_hashes)

24084

In [27]:
all_hashes['#trump']

[43, 133]

<h2>
Challenge 1 - we need some basic probabilities
</h2>
<div class=h1_cell>
<p>
We will need various counts and probabilities. Please calculate the values and put them in useful_counts. And don't just copy my values as constants!
<p>
</div>

In [76]:
# For P(E1|O) we need to know how many times the hashtag appears in nonhate tweets and how many times it appears in hate tweets. We can add those to get the total number of tweets it appears in.
# For P(O) we need the count of each type of tweet, nohate and hate. And we can add those to get the total number of tweets.
# For P(E0|O) we need the count of nohate tweets that have no hashtag and the count of hate tweets that have no hashtag. And we can add those to get the total.

useful_counts = {}

useful_counts['tweet_count'] = len(tweet_table)
useful_counts['class_count'] = [len(tweet_table)-sum(tweet_table['label']) , sum(tweet_table['label'])]
useful_counts['class_prob'] = [useful_counts['class_count'][0]/useful_counts['tweet_count'] , useful_counts['class_count'][1]/useful_counts['tweet_count']]

useful_counts

{'class_count': [29720, 2242],
 'class_prob': [0.92985420186471435, 0.070145798135285653],
 'tweet_count': 31962}

<h2>
Challenge 2 - E0 (AKA naked tweets)
</h2>
<div class=h1_cell>
<p>
We need a count of nohate tweets lacking a hashtag and hate tweets lacking a hashtag. Please put your info in `useful_counts`. And again, do your own calculation and don't just copy my constant values.
</div>

In [77]:
# filter out tweets with hash_count == 0
naked_tweets = tweet_table.loc[tweet_table['hash_count'] == 0]
useful_counts['naked_count'] = [len(naked_tweets)-sum(naked_tweets['label']) , sum(naked_tweets['label'])]

useful_counts

{'class_count': [29720, 2242],
 'class_prob': [0.92985420186471435, 0.070145798135285653],
 'naked_count': [7899, 607],
 'tweet_count': 31962}

<h2>
Challenge 3 - Let's put it together
</h2>
<div class=h1_cell>
<p>
Ok, I think we are ready for our naive_bayes function. Please fill it out and make sure you match my results.
<p>
But one important point: if you find a hashtag that is not in all_hashes, just skip over it. I'll say something more about this later.
<p>
Another important note. We will be using this same function next week on a whole new problem looking at regular old sentences (no hashtags involved). And we will have 3 possible classes, i.e., 0,1,2. So try hard not to tie your function directly to the 2-class tweet problem. I know this is a bit of a problem given our notion of naked tweets will not carry over to week 4. But see if you can make other portions of your code general.
</div>

In [39]:
hash_pat = r'[#][\w]+' # hmm... need to go back and test this against module 2 still (and if it's good, update all_hashes..)

# hash_pat = r'[#][\W]+' # hmm...
hash_patc = re.compile(hash_pat)

hash_patc.findall("yeah#y3s#YAYS! + an#DYa #no$") # just testing

['#y3s', '#YAYS', '#DYa', '#no']

In [129]:
def naive_bayes(text, count_dictionary, patc, bag_of_words, class_size):

    ##
    # --- Interpretation of variables in context of tweets: hate/nohate
    # text = I want to predict a category/class for
    # count_dictionary = useful counts obtained from a labled dataset table
    # patc = regex pattern for searching (hashtags)
    # bag_of_words = our table of all hashtags from the tweet_table
    # class_size = number of possible categories (hate or nohate)
    # NoHate = class 0 , Hate = class 1
    #
    # Will return list of probabilities for each class
    ##
    
    from functools import reduce # need to use to computing a product over a list 
    import re # for regex processes
        
    ### From text, extract all the hashtags and store in 'hashtags'
    patterns = re.compile(patc) # creates the regex pattern
    hashtags = patterns.findall(text) # gets all the hashtags from the text
    
    
    ### Check if it's a 'naked tweet' (no hashtags) --- Treat this as a Separate Case
    if hashtags == []: # naked tweet
        # Compute 'naked tweet' Class Probabilities
        naked_prob = []
        for i in range(0,class_size): # checks i=0,1,...,class_size-1
            naked_prob = naked_prob + [count_dictionary['naked_count'][i] / count_dictionary['tweet_count']] 
            # 'naked_count' and 'class_count' are hard-coded in useful_counts 
            
            # Note: at first I thought it might be 
            # naked_prob = naked_prob + [count_dictionary['naked_count'][i] / count_dictionary['class_count'][i]] 
            # but that skews the values by measuring the wrong proportions
            
        return naked_prob # the probabilities for different classes given a naked tweet
    
    else: ### the typical case
    
        ### Compute Background Class Probabilities: what will be P(nohate) - (OPTION 0) & P(hate) - (OPTION 1) 
        background_prob = []
        for i in range(0,class_size): # checks i=0,1,...,class_size-1
            background_prob = background_prob + [count_dictionary['class_prob'][i]] # 'class_prob' is hard-coded in useful_counts 


        ### Compute P(hashtags=evidence | Outcomes = classes)
        likelihoods = [[1] * class_size] # initializes list of sublists [ [*], [*], ... ]
        # these will hold the conditional probabilities P (Ei | O)
        # each sublist [*] will hold the P(Ei | O) given the single hashtage Ei 
        # with an entry for all the different classes O - i.e., [*] = [P("trump" | nohate) , P("trump" | hate)]
        # likelihoods initialized as [ [1,1,...,1] ]. Since we will multiply over it
        # 'default information' = entry of 1 is fine, won't modify probabilities
        # other option is initialize as empty.. but problems computing probabilities then
        # this way we avoid edge case of no tweets from table.    

        for entry in hashtags: # loop over hashtags

            if entry not in bag_of_words:
                pass # do nothing - we won't count a hashtag if it's not in our list yet
            else: 
                new_probabilities = [] # initialize a sublist [*]
                for i in range(0,class_size): # iterate over the classes
                    new_probabilities = new_probabilities + [bag_of_words[entry][i] / count_dictionary['class_count'][i]]
                    # 

                likelihoods.append(new_probabilities) #add a sublist of P(Ei|O) for fixed Ei and all classes O to likelihoods

        ### Compute class probabilities given evidence: P(O | E1 ... Ek)
        # we ignore P(E1 ... Ek) normalization factor since it's the same for both terms & so doesn't influence comparison
        probabilities = [] 

        for i in range(0,class_size): # multiply all corresponding probabilities for each class
            probabilities = probabilities + [ background_prob[i] * reduce(lambda x, y: x * y, [item[i] for item in likelihoods], 1) ]
        # looking around online, we can probably slightly optimize above by using different function than reduce...

        return probabilities    
    

In [130]:
naive_bayes(tweet_table.loc[0, 'tweet'], useful_counts, hash_patc, all_hashes, 2)  # actual 0 - nohate

[0.0010324760653275765, 0.0]

<div class=h1_cell>
<p>
Wow. Pretty dang certain that the label is not 1 for row0.
<p>
Think a bit about how we could get a 0.0 value for hate. I'll bring the formula down here so we can study it. Fill in the O with hate.
<p>
<img src='https://www.dropbox.com/s/gstzvvtvh9b39o8/bayes.png?raw=1'>
<p>
If `P(hate)` was 0 that would do it. But seems kind of unlikely that we have zero hate tweets. That means one or more of the `P(hashtag|hate)` values was 0. In essence, one or more of the hashtags in row0 never appears in a hate tweet.
<p>
Let's try another.
</div>

In [131]:
naive_bayes(tweet_table.loc[13, 'tweet'], useful_counts, hash_patc, all_hashes, 2)   # actual 1 - hate tweet

[1.062648656237095e-12, 1.5685392010692217e-09]

<div class=h1_cell>
<p>
Got it right again.
<p>
Let's look at the problem of seeing unknown hashtags, i.e., ones not in all_hashes. We can test our code out to make sure things are handled correctly.
</div>

In [132]:
'#stevef' in all_hashes  # so #stevef is new tag not seen previously.

False

<div class=h1_cell>
<p>
I said to skip over unknown hashtags so we should skip over #stevef.
</div>

In [133]:
test1 = naive_bayes('here is a new tweet by #stevef', useful_counts, hash_patc, all_hashes, 2) # skip unknown but don't treat as naked tweet

test1

[0.92985420186471435, 0.070145798135285653]

<div class=h1_cell>
<p>
Above is just (P(nohate), P(hate)) which is same as general class probabilities.
</div>

In [134]:
test1 == useful_counts['class_prob']

True

<div class=h1_cell>
<p>
Ok, let's test it with a naked tweet - no hashtags.
</div>

In [136]:
test2 = naive_bayes('here is a new tweet by stevef', useful_counts, hash_patc, all_hashes, 2)  #here is naked tweet
test2

[0.24713722545522809, 0.018991302171328453]

<div class=h1_cell>
<p>
Looks like now treating it as a naked tweet and rightfully so.
</div>

In [147]:
# test2 == tuple(1.0*x/useful_counts['tweet_count'] for x in useful_counts['naked_count'])
test2 == [1.0*x/useful_counts['tweet_count'] for x in useful_counts['naked_count']]


# note for myself:
# initially I measured this for the 'alternate' normalization factors (which were wrong) 
# test2 == [1.0*useful_counts['naked_count'][i]/useful_counts['class_count'][i] for i in list(range(len(useful_counts['class_count'])))]

True

<div class=h1_cell>
<p>
I am saying that a tweet with a hashtag is not a naked tweet, even if we do not recognize the hashtag. Seems reasonable to me. But more generally, my choice of skipping over unknowns is semi-arbitrary. I'll give you another approach later.
</div>

<h2>
Ok, I think we can do this quickly
</h2>
<div class=h1_cell>
<p>
But I'll still time it just in case. I want the predictions for all the tweets.
</div>

In [138]:
import time

In [139]:
start = time.time()

predictions = []
for i,row in tweet_table.iterrows():
    if i%1000 == 0: print('did 1000')
    pair = naive_bayes(row['tweet'], useful_counts, hash_patc, all_hashes, 2)
    predictions.append(0 if pair[0] >= pair[1] else 1)
    
end = time.time()
print(end - start)  # in seconds

## about 2.5 seconds... ~3x as fast as yours? I'm a little worried at that kind of speedup...

did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
did 1000
2.4525539875030518


<h2>
Don't know about you, but I think we won big
</h2>
<div class=h1_cell>
<p>
I think pre-building the hashtag counts dictionary gave us a huge speed-up. 7 seconds for 32K tweets.
</div>

In [140]:
predictions[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

<h2>
Challenge 4 - compute the confusion matrix
</h2>
<div class=h1_cell>
<p>
Generate the confusion matrix (as a dictionary) so we can check out how we are doing. And once again, do not simply copy my values - do your own computation.
</div>

In [141]:
actuals = tweet_table['label']  #easy peasy to pull a column out of a table
zipped = zip(predictions,actuals)

In [142]:
confusion_dictionary = {(1,1):0, (1,0):0, (0,1):0, (0,0):0}
for pair in zipped:
    confusion_dictionary[pair] += 1
confusion_dictionary

{(0, 0): 29631, (0, 1): 697, (1, 0): 89, (1, 1): 1545}

<h2>
Kind of interesting at first glance
</h2>
<div class=h1_cell>
<p>
We are making 90 errors on predicting hate when it is not. But 703 errors predicting non-hate when it was hate.
<p>
How should we feel about that? If you are a free-speecher, you might be happy. We are erring on side of letting things through. If you are a parent, you might prefer more (1,0) than (0,1).
</div>

In [144]:
accuracy = (1.0*confusion_dictionary[(0,0)]+confusion_dictionary[(1,1)])/len(tweet_table)
accuracy

# compare against your 0.9751892872786434

0.9754082973531069

<div class=h1_cell>
Let's check this against what we would get if we always predicted 0 (nohate).
</div>

In [145]:
useful_counts['class_prob'][0]

0.92985420186471435

<div class=h1_cell>
Not bad. We jumped 5 percent. And this is a hard problem because there are so few hate tweets in the overall set.
</div>

<h2>
Why can we throw away the denominator?
</h2>
<div class=h1_cell>
<p>
Our prediction rule is calculate P(nohate|E1, E2, ...Ek) and P(hate|E1, E2, ...Ek). Then choose the larger. Using Naive Bayes theorem we see that the denominator P(E1) `*` P(E2) ... `*` P(Ek) is same for both; the denominator does not depend on the case we are looking at. It will not affect the outcome. Throw it out.
<p>
BTW: I would argue along a similar line for euclidean distance. Suppose I have ed(v0,v1) &lt; ed(v0,v2). The last thing we do in the ed function is take the square root. What if I eliminated that step. Would it change the outcome of the comparison? Nope.
</div>

<h2>
Why is Bayes Naive?
</h2>
<div class=h1_cell>
<p>
Well, Bayes, himself, was not naive. Or maybe he was, don't know. What is meant by naive is an assumption we make to allow us to use the simplified formula we used above.
Let's look again at row13 from week 2. The non-naive way of looking at it is `P(hate, #cnn, #michigan, #tcot)` where we view all terms as dependent on each other, e.g. we can't view #cnn on its own but only in conjunction with (conditional on) #michigan and #tcot. The probability-chain rule formula captures this idea:
<p>
<img src='https://www.dropbox.com/s/v2ppatadfghvrb4/chain2.png?raw=1'>
<p>
All of these probabilities are known to us so we could calculate an answer. Our current approach tries to avoid calculating much at prediction time. Instead we pre-build our dictionary and store the counts there. Computation cost is just look up cost in a Python dictionary (fast). Can we use the same approach with the formula above? We would need a dictionary that contains all possible combinations of hashtags with their counts, i.e., the powerset. We have 23K unique hastags. The size of the powerset is `2**N`. So we are looking at a dictionary with 2**23000 items. Gulp. That's not going to happen. Our alternative is to forget the dictionary and just do on-demand counting for each new tweet and set of hashtags. But this could be slow if we have to search the entire tweet_table multiple times for every new tweet. And if you are an app developer with a cool new tweet-filter, you want this happening in real-time as tweets come in.
<p>To get around the space and computational costs of the formula above (the full chain rule), Smith said let's pretend that all terms are independent of each other. Others said, don't be naive, Smith, you can't assume that terms are totally independent. But Smith persevered. And she was right, the naive assumption does work well in many cases. So Naive Bayes is normal Bayes but with the strong (naive) assumption that all items are independent.
<p>
Who was Smith? I don't have a definitive answer. But I like her thinking. Try simple first.
</div>

<h2>
Can we find an alternative to skipping unknowns?
</h2>
<div class=h1_cell>
<p>
Someone can always come up with an alternative in probabilistic reasoning :)
<p>
Let's say we don't skip unknown hashtags. So we treat things like `#stevef` as probability 0 for both cases (!) Of course, this would lead to a probability of nohate as 0 and a probability of hate as 0. Not so good. To get around that, we add a smoothing factor to P(Ei|O). 
<p>
<img src='https://www.dropbox.com/s/ejp4nm1y3uldax2/smoothng2.png?raw=1'>
<p>
The # symbol stands for count above. The value of k is your choice. In fact, you could experiment with different values to get the best choice. The value of n sub i is the size of all_hashes. As an example, if you chose k to be .001, we would get the following:
<p>
<code>
P(#stevef | nohate) = (0 + .001)/(total_counts[0] + .001*23013)
</code>
<p>
There are a lot of interesting variations on this style of smoothing (called Laplace Smoothing). And the cool thing is we have 2 faculty that do research with probabilistic reasoning. I encourage you to take a course from Daniel Lowd and Thanh Nguyen to get a deeper view into the problem. We just hired Thanh so you can also welcome her to the department!
</div>

<h2>
Wrangling the table
</h2>
<div class=h1_cell>
<p>
Before you leave, write the tweet_table out to file. We did a lot of wrangling on it so let's save what we have done.
</div>

In [148]:
week = '3' # change this each version

home_path =  os.path.expanduser('~')

file_path = '/Documents/datascience_2/table_history/'  #use your own path

file_name = 'tweet_table_w'+week+'.csv'

tweet_table.to_csv(home_path + file_path + file_name, index=False)